In [1]:
import random
import mysql.connector
import json
import string
import datetime as dt
from pymongo import MongoClient

def getGenero(argument):
    switcher = {        
        5: "action",
        6: "adventure",
        7: "casual",
        8: "strategy",
        9: "RPG",
        10: "simulation",
        11: "early_access",
        12: "free_to_play",
        13: "sports",
        14: "racing",
        15: "massively_multiplayer"
    }
    return switcher.get(argument, argument)

myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient['stum_for_you']
mycol = mydb["usuarios"]
colmul = mydb["multidimensional"]

mydb = mysql.connector.connect(
    host="localhost",
    user="root",
    database='stum_for_you',
    passwd=""
)
from datetime import datetime,timedelta
hoy = datetime.now()
dias = timedelta(days=7)
semana_pasada = hoy-dias  
#se comprueba que sea domingo (0)
comp=hoy.strftime("%w")
print("Día de la semana:",comp)
if (comp == '4') :#--------------------------------------------------------------cuidado
    esjuego=False
    esDLC=False
    hoy=hoy.strftime("%Y-%m-%d")
    #semana_pasada=semana_pasada.strftime("%Y:%m:%d")
    
    semana_pasada ="2017-01-01"#-------------------------------------------------para pruebas
    print(hoy,semana_pasada)
    mycursor = mydb.cursor(buffered=True)

    mycursor.execute("SELECT * FROM transacciones_compra WHERE fecha_compra BETWEEN %s AND  %s", (semana_pasada,hoy,))
    transacciones = mycursor.fetchall()
    tipo_movimiento = "compra"
    for t in transacciones :
   
    #"movimiento"
        precio_final = t[1]
        mycursor.execute("SELECT * FROM compra_juegos WHERE id_transaccion = %s", (t[0],))
        compra = mycursor.fetchall()
        cantidad = len(compra)
    #fecha
        anyo=t[2].strftime("%Y")
        mes=t[2].strftime("%m")
        dia=t[2].strftime("%d")
    #juego
        if(len(compra)>0) :
            esjuego=True
            esDLC=False
            mycursor.execute("SELECT id_juego FROM claves_juegos WHERE id_clave = %s", (compra[0][2],))
            puntero_juego = mycursor.fetchone()[0]
            mycursor.execute("SELECT * FROM juegos WHERE id_juego = %s", (puntero_juego,))
            juego = mycursor.fetchone()
            genero= []
            #sacar genero
            for a in range (5,15) :
                if (juego[a]==True) :
                    genero.append(getGenero(a))
            precio_juego=juego[3]
            clasificacion=juego[1]
     #dlc
        if(len(compra)<=0) :
            print("***********************ha entrado en dlc***********************************")
            esDLC=True
            esjuego=False
            print(t[0])
            mycursor.execute("SELECT * FROM compra_dlcs WHERE id_transaccion = %s", (t[0],))
            compra = mycursor.fetchall()
            cantidad = len(compra)
            print(compra)
            mycursor.execute("SELECT id_dlc FROM claves_dlc WHERE id_clave = %s", (compra[0][2],))
            puntero_dlc = mycursor.fetchone()[0]
            mycursor.execute("SELECT * FROM dlcs WHERE id_dlc = %s", (puntero_dlc,))
            dlc = mycursor.fetchone()
            tipo= dlc[3]
            precio_dlc=dlc[1]
            
    #proveedor
        puntero_proveedor=compra[0][0]
        mycursor.execute("SELECT * FROM proveedor WHERE id_proveedor = %s", (puntero_proveedor,))
        provedor = mycursor.fetchone()
        pais= provedor[3]
        valoracion=provedor[1]
        
    #insertar datos en multidimensional    
        if esjuego:
            datos = {
                "movimiento":{"precio_final":precio_final, "tipo_movimiento": tipo_movimiento, "cantidad": cantidad},
                "fecha":{"anyo":anyo, "mes":mes,"dia":dia},
                "juego":{"puntero_juego":puntero_juego,"genero":genero,"precio_juego":precio_juego,"clasificacion":clasificacion},
                "proveedor":{"puntero_proveedor":puntero_proveedor,"pais":pais,"valoracion":valoracion}
             }
        elif esDLC:
             datos = {
                "movimiento":{"precio_final":precio_final, "tipo_movimiento": tipo_movimiento, "cantidad": cantidad},
                "fecha":{"anyo":anyo, "mes":mes,"dia":dia},
                "dlc":{"puntero_dlc":puntero_dlc,"tipo":tipo,"precio_dlc":precio_dlc},
                "proveedor":{"puntero_proveedor":puntero_proveedor,"pais":pais,"valoracion":valoracion}
             }
        print("---------------------------------------------------------------")        
        print(datos)
        x = colmul.insert_one(datos) 
        
    

Día de la semana: 4
2020-01-16 2017-01-01
---------------------------------------------------------------
{'movimiento': {'precio_final': 5, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '02', 'dia': '05'}, 'juego': {'puntero_juego': 7066, 'genero': ['adventure', 'casual'], 'precio_juego': 7, 'clasificacion': 'AAA'}, 'proveedor': {'puntero_proveedor': 14, 'pais': 'Virgin Islands, British', 'valoracion': 2}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 8, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '06', 'dia': '07'}, 'juego': {'puntero_juego': 5826, 'genero': [], 'precio_juego': 10, 'clasificacion': 'INDIE'}, 'proveedor': {'puntero_proveedor': 34, 'pais': 'Niue', 'valoracion': 1}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 20, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '12', 'dia': '3

---------------------------------------------------------------
{'movimiento': {'precio_final': 16, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '03', 'dia': '16'}, 'juego': {'puntero_juego': 1071, 'genero': ['action', 'adventure'], 'precio_juego': 20, 'clasificacion': 'AAA'}, 'proveedor': {'puntero_proveedor': 34, 'pais': 'Niue', 'valoracion': 1}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 0, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '07', 'dia': '09'}, 'juego': {'puntero_juego': 5378, 'genero': [], 'precio_juego': 0, 'clasificacion': 'INDIE'}, 'proveedor': {'puntero_proveedor': 18, 'pais': 'China', 'valoracion': 3}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 0, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '12', 'dia': '08'}, 'juego': {'puntero_juego': 10975, 'genero': ['action', 

---------------------------------------------------------------
{'movimiento': {'precio_final': 24, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '03', 'dia': '16'}, 'juego': {'puntero_juego': 4674, 'genero': ['strategy'], 'precio_juego': 30, 'clasificacion': 'AAA'}, 'proveedor': {'puntero_proveedor': 23, 'pais': 'Estonia', 'valoracion': 7}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 4, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2020', 'mes': '01', 'dia': '16'}, 'juego': {'puntero_juego': 6556, 'genero': ['action', 'adventure'], 'precio_juego': 5, 'clasificacion': 'INDIE'}, 'proveedor': {'puntero_proveedor': 35, 'pais': 'British Indian Ocean Territory', 'valoracion': 8}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 16, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '02', 'dia': '19'}, 'juego': {'punte

---------------------------------------------------------------
{'movimiento': {'precio_final': 8, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '02', 'dia': '22'}, 'juego': {'puntero_juego': 208, 'genero': ['action', 'strategy', 'simulation'], 'precio_juego': 10, 'clasificacion': 'INDIE'}, 'proveedor': {'puntero_proveedor': 27, 'pais': 'Sweden', 'valoracion': 9}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 12, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '05', 'dia': '29'}, 'juego': {'puntero_juego': 8133, 'genero': ['action', 'early_access'], 'precio_juego': 15, 'clasificacion': 'INDIE'}, 'proveedor': {'puntero_proveedor': 7, 'pais': 'Canada', 'valoracion': 8}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 120, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '10', 'dia': '02'}, 'juego': {'p

---------------------------------------------------------------
{'movimiento': {'precio_final': 8, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '08', 'dia': '28'}, 'juego': {'puntero_juego': 763, 'genero': ['strategy'], 'precio_juego': 10, 'clasificacion': 'AAA'}, 'proveedor': {'puntero_proveedor': 39, 'pais': 'Western Sahara', 'valoracion': 5}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 0, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '05', 'dia': '17'}, 'juego': {'puntero_juego': 12326, 'genero': ['RPG', 'free_to_play'], 'precio_juego': 0, 'clasificacion': 'AAA'}, 'proveedor': {'puntero_proveedor': 20, 'pais': 'Sierra Leone', 'valoracion': 8}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 4, 'tipo_movimiento': 'compra', 'cantidad': 1}, 'fecha': {'anyo': '2019', 'mes': '06', 'dia': '21'}, 'juego': {'puntero_juego': 8543

...

---------------------------------------------------------------
{'movimiento': {'precio_final': 285, 'tipo_movimiento': 'compra', 'cantidad': 57}, 'fecha': {'anyo': '2017', 'mes': '01', 'dia': '31'}, 'juego': {'puntero_juego': 9114, 'genero': ['adventure'], 'precio_juego': 10, 'clasificacion': 'AAA'}, 'proveedor': {'puntero_proveedor': 27, 'pais': 'Sweden', 'valoracion': 9}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 244, 'tipo_movimiento': 'compra', 'cantidad': 61}, 'fecha': {'anyo': '2018', 'mes': '04', 'dia': '04'}, 'juego': {'puntero_juego': 10023, 'genero': ['action', 'early_access'], 'precio_juego': 10, 'clasificacion': 'INDIE'}, 'proveedor': {'puntero_proveedor': 8, 'pais': 'Afghanistan', 'valoracion': 2}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 455, 'tipo_movimiento': 'compra', 'cantidad': 91}, 'fecha': {'anyo': '2017', 'mes': '02', 'dia': '09'}, 'juego': {'puntero_juego

---------------------------------------------------------------
{'movimiento': {'precio_final': 1242, 'tipo_movimiento': 'compra', 'cantidad': 69}, 'fecha': {'anyo': '2018', 'mes': '03', 'dia': '04'}, 'juego': {'puntero_juego': 8093, 'genero': ['simulation'], 'precio_juego': 20, 'clasificacion': 'AAA'}, 'proveedor': {'puntero_proveedor': 8, 'pais': 'Afghanistan', 'valoracion': 2}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 160, 'tipo_movimiento': 'compra', 'cantidad': 80}, 'fecha': {'anyo': '2018', 'mes': '07', 'dia': '09'}, 'juego': {'puntero_juego': 11490, 'genero': ['strategy', 'simulation'], 'precio_juego': 10, 'clasificacion': 'INDIE'}, 'proveedor': {'puntero_proveedor': 15, 'pais': 'Nepal', 'valoracion': 10}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 2400, 'tipo_movimiento': 'compra', 'cantidad': 60}, 'fecha': {'anyo': '2019', 'mes': '07', 'dia': '23'}, 'juego': {'puntero_ju

---------------------------------------------------------------
{'movimiento': {'precio_final': 0, 'tipo_movimiento': 'compra', 'cantidad': 80}, 'fecha': {'anyo': '2019', 'mes': '07', 'dia': '26'}, 'juego': {'puntero_juego': 8930, 'genero': ['adventure'], 'precio_juego': 2, 'clasificacion': 'INDIE'}, 'proveedor': {'puntero_proveedor': 26, 'pais': 'Laos', 'valoracion': 6}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 228, 'tipo_movimiento': 'compra', 'cantidad': 57}, 'fecha': {'anyo': '2018', 'mes': '05', 'dia': '04'}, 'juego': {'puntero_juego': 7959, 'genero': ['adventure'], 'precio_juego': 10, 'clasificacion': 'INDIE'}, 'proveedor': {'puntero_proveedor': 20, 'pais': 'Sierra Leone', 'valoracion': 8}}
---------------------------------------------------------------
{'movimiento': {'precio_final': 73, 'tipo_movimiento': 'compra', 'cantidad': 73}, 'fecha': {'anyo': '2019', 'mes': '12', 'dia': '31'}, 'juego': {'puntero_juego': 4416, 'genero

---------------------------------------------------------------
{'movimiento': {'precio_final': 540, 'tipo_movimiento': 'compra', 'cantidad': 90}, 'fecha': {'anyo': '2018', 'mes': '06', 'dia': '06'}, 'dlc': {'puntero_dlc': 239, 'tipo': ' cosmetico ', 'precio_dlc': 17}, 'proveedor': {'puntero_proveedor': 32, 'pais': 'Angola', 'valoracion': 3}}
***********************ha entrado en dlc***********************************
3609
[(6, 3609, 1507, 8), (6, 3609, 1508, 8), (6, 3609, 1509, 8), (6, 3609, 1510, 8), (6, 3609, 1511, 8), (6, 3609, 1512, 8), (6, 3609, 1513, 8), (6, 3609, 1514, 8), (6, 3609, 1515, 8), (6, 3609, 1516, 8), (6, 3609, 1517, 8), (6, 3609, 1518, 8), (6, 3609, 1519, 8), (6, 3609, 1520, 8), (6, 3609, 1521, 8), (6, 3609, 1522, 8), (6, 3609, 1523, 8), (6, 3609, 1524, 8), (6, 3609, 1525, 8), (6, 3609, 1526, 8), (6, 3609, 1527, 8), (6, 3609, 1528, 8), (6, 3609, 1529, 8), (6, 3609, 1530, 8), (6, 3609, 1531, 8), (6, 3609, 1532, 8), (6, 3609, 1533, 8), (6, 3609, 1534, 8), (6, 3609, 153

[(18, 3661, 3606, 13), (18, 3661, 3607, 13), (18, 3661, 3608, 13), (18, 3661, 3609, 13), (18, 3661, 3610, 13), (18, 3661, 3611, 13), (18, 3661, 3612, 13), (18, 3661, 3613, 13), (18, 3661, 3614, 13), (18, 3661, 3615, 13), (18, 3661, 3616, 13), (18, 3661, 3617, 13), (18, 3661, 3618, 13), (18, 3661, 3619, 13), (18, 3661, 3620, 13), (18, 3661, 3621, 13), (18, 3661, 3622, 13), (18, 3661, 3623, 13), (18, 3661, 3624, 13), (18, 3661, 3625, 13), (18, 3661, 3626, 13), (18, 3661, 3627, 13), (18, 3661, 3628, 13), (18, 3661, 3629, 13), (18, 3661, 3630, 13), (18, 3661, 3631, 13), (18, 3661, 3632, 13), (18, 3661, 3633, 13), (18, 3661, 3634, 13), (18, 3661, 3635, 13), (18, 3661, 3636, 13), (18, 3661, 3637, 13), (18, 3661, 3638, 13), (18, 3661, 3639, 13), (18, 3661, 3640, 13), (18, 3661, 3641, 13), (18, 3661, 3642, 13), (18, 3661, 3643, 13), (18, 3661, 3644, 13), (18, 3661, 3645, 13), (18, 3661, 3646, 13)]
---------------------------------------------------------------
{'movimiento': {'precio_final': 5